## Detection on Static Image

### Import TensorFlow and required packages.

In [2]:
pip install tensorflow_hub

     -------------------------------------- 90.3/90.3 kB 727.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow_hub as hub #to load object detection model from tensorflow hub
import cv2 #for image processing
import numpy # for numerical calculation
import pandas as pd #to load CSV text files
import tensorflow as tf
import matplotlib.pyplot as plt #to plot images

## Model

In [4]:
#load pre-trained model and labels
print("[INFO] Loading Mode.....")
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")
print("[INFO] Finished Model Loading....")

[INFO] Loading Mode.....
[INFO] Finished Model Loading....


In [12]:
#load csv file which contains labels of the multiple classes
labels = pd.read_csv('labels.csv', sep=';', index_col="ID")
labels = labels['OBJECT (2017 REL.)']

#let's check first five labels of the classes
labels.head()

FileNotFoundError: [Errno 2] No such file or directory: 'labels.csv'

## Load the Image
Firstly, we load the example image and process it to the expected format for the TensorFlow model

In [7]:
# set dimension of image to resize
width = 1028
height = 1028

# load image by OpenCV
img = cv2.imread('images\example_01.jpg')

#resize of respect the input_shpae
inp = cv2.resize(img, (width, height))

#convert img to rgb
rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)

#convert img to tensor image
rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.unit8)
print("Before : ", rgb_tensor.shape)

#add dims to rgb tensor
rgb_tensor = tf.expend_dims(rgb_tensor, 0)
print("After : ", rgb_tensor.shape)

plt.figure(figsize=(3,3))
plt.axis("off")
plt.imshow(rgb)


## Perform Detection

The model will return:
    (1) The bounding box boundaries of the detection
    (2) The detection scores (Probabilities of a given class);
    (3) The detection classes;
    (4) The number of detections

In [ ]:
#the output dictionaries of the detector
boxes, scores, classes, num_detections = detector(rgb_tensor)

#get the predicted labels
pred_labels = classes.numpy().astype('int')[0]
pred_labels = [labels[i] for i in pred_labels]

#get the predicted bounding boxes location
pred_boxes = boxes.numpy()[0].astype('int')

#get the predicted scores
pred_scores = scores.numpy()[0]

#copy the input image to draw bounding boxes
for score, (ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
    #set the minimum confidence score as 0.5
    if score < 0.56:
        continue
    
    text = "{} : {:.2f}%".format(label, score*100)
    cv2.rectangle(result, (xmin,ymax),(xmax,ymin),(0,255,0),3)
    y = ymin - 20 if ymin - 20 > 20 else ymin + 20
    cv2.putText(result, text, (xmin, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 3)
    
#show the result image
plt.figure(figsize=(8,8))
plt.axis("off")
plt.imshow(result)
    